## Import Packages and Set Directory

In [55]:
import pandas as pd
import numpy as np
import sys
import argparse
import os
os.chdir('/Users/abc6435/Desktop/RABvcfs')

## Read VCF

In [99]:
#Read VCF
def readvcf(vcf_file, pop_file):
    #Import Population File
    pops = pd.read_csv(pop_file, sep="\t", header=None)
    popA = pops[pops[0]=="popA"][1].astype(str)
    popB = pops[pops[0]=="popB"][1].astype(str)
    print("Population A \n", popA)
    print("Population B \n", popB)

    #Import VCF
    with open(vcf_file) as file:
        for line in file:
            if line.startswith('#CHROM'):
                cols = line.lstrip("#").strip().split("\t")
    data = pd.read_csv(vcf_file, sep='\t', comment='#', names=cols)

    #Extract Genotype Information
    loci = data[['CHROM','POS','REF','ALT']]
    samples = data.drop(columns=['CHROM', 'POS', 'REF', 'ALT','ID','QUAL','FILTER','INFO','FORMAT'])
    samples = samples.apply(lambda col: col.str.split(":").str[0])
    genotypes = pd.concat([loci,samples], axis=1)

    #Count Genotypes
    genotypes['popA_0'] = 0
    genotypes['popA_1'] = 0
    genotypes['popB_0'] = 0
    genotypes['popB_1'] = 0

    for sample in popA:
        for row in range(len(genotypes)):
            if genotypes.loc[row, sample] == '0/0' or genotypes.loc[row, sample] == '0|0':
                genotypes.loc[row, 'popA_0'] += 2
            if genotypes.loc[row, sample] == '1/1' or genotypes.loc[row, sample] == '1|1':
                genotypes.loc[row, 'popA_1'] += 2
            if genotypes.loc[row, sample] == '0/1' or genotypes.loc[row, sample] == '0|1':
                genotypes.loc[row, 'popA_0'] += 1
                genotypes.loc[row, 'popA_1'] += 1
            if genotypes.loc[row, sample] == '1/0' or genotypes.loc[row, sample] == '1|0':
                genotypes.loc[row, 'popA_0'] += 1
                genotypes.loc[row, 'popA_1'] += 1
                
    for sample in popB:
        for row in range(len(genotypes)):
            if genotypes.loc[row, sample] == '0/0' or genotypes.loc[row, sample] == '0|0':
                genotypes.loc[row, 'popB_0'] += 2
            if genotypes.loc[row, sample] == '1/1' or genotypes.loc[row, sample] == '1|1':
                genotypes.loc[row, 'popB_1'] += 2
            if genotypes.loc[row, sample] == '0/1' or genotypes.loc[row, sample] == '0|1':
                genotypes.loc[row, 'popB_0'] += 1
                genotypes.loc[row, 'popB_1'] += 1
            if genotypes.loc[row, sample] == '1/0' or genotypes.loc[row, sample] == '1|0':
                genotypes.loc[row, 'popB_0'] += 1
                genotypes.loc[row, 'popB_1'] += 1

   #Calculate Frequencies
    genotypes['f_popA_0'] = genotypes['popA_0']/(genotypes['popA_0'] + genotypes['popA_1'])
    genotypes['f_popA_1'] = genotypes['popA_1']/(genotypes['popA_0'] + genotypes['popA_1'])
    genotypes['f_popB_0'] = genotypes['popB_0']/(genotypes['popB_0'] + genotypes['popB_1'])
    genotypes['f_popB_1'] = genotypes['popB_1']/(genotypes['popB_0'] + genotypes['popB_1'])
    genotypes['nA'] = (genotypes['popA_0'] + genotypes['popA_1']) / 2
    genotypes['nB'] = (genotypes['popB_0'] + genotypes['popB_1']) / 2
    freq = genotypes[['CHROM','POS','REF','ALT',
                      'f_popA_1','f_popB_1','nA','nB']].dropna(axis="rows")
    return freq[(freq["nA"] > 1) & (freq["nB"] > 1)]

## Import Sites

In [ ]:
def importsites(neutral_sites, mutation_sites:
    neutral = pd.read_csv(neutral_sites, sep='\t', header=(0))
    mutation = pd.read_csv(mutation_sites, sep='\t', header=(0))
    if not {'chromo', 'position'}.issubset(neutral.columns):
        sys.exit("ERR: Check file headers.")
    if not {'chromo', 'position'}.issubset(mutation.columns):
        sys.exit("ERR: Check file headers")

Population A 
 7      29779
8     383194
9     383202
10    383205
11    507264
12    507265
Name: 1, dtype: object
Population B 
 0    183195332
1    183194861
2    183195321
3    183195304
4    183194841
5    183195326
6    183195312
Name: 1, dtype: object
